# Random Forest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
from collections import defaultdict 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.ensemble import RandomForestClassifier

# Du precedent projet
import randomforest as rf 

## 1. Importation des données

In [2]:
data = pd.read_csv("data.csv")

In [ ]:
print(data.shape)
data.head()

In [3]:
# On supprime la premiere colonne
data = data.drop(['filename'],axis=1)

# On change les noms des genres par des entiers (de 0 a 9) car notre random forest ne prend en compte que des entiers comme labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
labels = encoder.fit_transform(genre_list)
data.iloc[:, -1] = labels

In [4]:
# On normalise le dataset
scaler = StandardScaler()
data = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## 2. Réglage des hyperparamètres
Nous nous sommes fortement inspiré de la méthode décrite dans cet article [W. Koehrsen. Hyperparameter Tuning the Random Forest in Python, Janv. 2018](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

Notre modèle a cinq paramètres, dont quatre que nous souhaitons optimiser  :
   - `n_trees` -- le nombre d'arbres de la forêt
   - `n_samples` -- le nombre de données à placer dans le noeud de chaque arbre avant qu'il ne soit partitionné
   - `n_cuts` -- le nombre de coupes à tester pour trouver la meilleure
   - `max_depth` -- la profondeur maximale de chaque arbre

Pour avoir une première idée de la meilleure combinaison d'hyperparamètres, nous allons effectuer une ... (Random Search Cross Validation). Cela consiste à tester un large choix de combinaisons qui ont été formées en tirant aléatoirement des valeurs dans une grille d'hyperparamètres.

### 2.1 Random Search Cross Validation

#### Random Hyperparameter Grid

On définit la grille pour la recherche aléatoire (Random Hyperparameter Grid) :

In [5]:
# n_trees
n_trees = [int(x) for x in np.linspace(start = 200, stop = 800, num = 10)] 

# n_samples
n_samples = [int(x) for x in np.linspace(start = 200, stop = 700, num = 10)]

# n_cuts 
# dans quel intervalle pourrait on tester ??
n_cuts = [int(x) for x in np.linspace(start = 10, stop = 100, num = 5)] 

# max_depth
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
# Est-ce qu'on ajoute un None comme dans l'article ? A voir selon notre random forest, est-ce qu'elle prend en compte
# un arg None pour max_depth ?

# Creation de la grille
random_grid = {'n_trees': n_trees,
                'n_samples': n_samples,
                'n_cuts': n_cuts,
                'max_depth': max_depth}

from pprint import pprint
print("Grille d'hyperparametres :\n")
pprint(random_grid)

Grille d'hyperparametres :

{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'n_cuts': [10, 32, 55, 77, 100],
 'n_samples': [200, 255, 311, 366, 422, 477, 533, 588, 644, 700],
 'n_trees': [200, 266, 333, 400, 466, 533, 600, 666, 733, 800]}


Au lieu de tester 10 * 10 * 5 * 10 = 5000 combinaisons d'hyperparamètres, nous allons seulement en sélectionner quelques unes aléatoirement.

#### Random Search Training

On construit tout d'abord le modèle de base.

In [6]:
# On separe le dataset en train set et test set (80%/20%)
data_train, data_test, label_train, label_test = train_test_split(data, labels, test_size=0.2)

In [7]:
our_rf = rf.OurRandomForestClassifier() # choix aléatoire
our_rf.get_params().keys()

dict_keys(['max_depth', 'n_cuts', 'n_samples', 'n_trees'])

On procède à la recherche randomisée sur 50 combinaisons, en utilisant une 3-fold CV

In [8]:
# Definition de la recherche randomisee
rf_random = RandomizedSearchCV(estimator = our_rf, 
                               param_distributions = random_grid, 
                               n_iter = 50, 
                               cv = 3, 
                               verbose = 1, 
                               random_state = 8)
# je dois encore comprendre les differents parametres

# Entrainement du modele
rf_random.fit(data_train, label_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


TypeError: only size-1 arrays can be converted to Python scalars

Les resultats

In [ ]:
print("La meilleure combinaison d'hyperparametres avec la recherche randomisee est :")
print(rf_random.best_params_)
print("")
print("Le score moyen du modele avec ces hyperparametres est :")
print(rf_random.best_score_)

### 2.2 Grid Search Cross Validation
Une fois qu'on connait a peu pres les meilleurs hyper-parametres
Plus d'aleatoire, on teste toutes les combinaisons

## 3. Apprentissage
On reprend le modèle que nous avions implementé lors du précédent projet.

In [ ]:
# On separe le dataset en train set et test set (80%/20%)
data_train, data_test, label_train, label_test = train_test_split(data, labels, test_size=0.2)

# data_train {numpy.ndarray} (800,30) 
# label_train {numpy.ndarray} (800,)

In [ ]:
# Creation de la random forest
# par defaut n_trees=100, n_samples = 100, n_cuts = 10, max_depth = 15
rf_classifier = rf.OurRandomForestClassifier() 

# Entrainement de la random forest
rf_classifier.fit(data_train, label_train)

In [ ]:
guessed_label = rf_classifier.predict(data_test[1,:])
guessed_genre = encoder.inverse_transform(np.array([guessed_label]))
print("La musique caracterisee par {0} a ete categorisee en {1}".format(data_test[1,:], guessed_genre))

Essayons maintenant sur l'ensemble des données de `data_train`

In [ ]:
guesses = [rf_classifier.predict(data_test[i,:]) for i in range(data_test.shape[0])]

print("Our function made good predictions at ", rf_classifier.score(guesses, label_test)*100, "% rate")

sklearn_rf = RandomForestClassifier(n_estimators=100, max_depth=15, max_features='sqrt')
sklearn_rf.fit(data_train, label_train)
guessess = sklearn_rf.predict(data_test)

print("Sklearn function made good predictions at ", rf_classifier.score(guessess, label_test)*100,"% rate")